In [1]:
open System
open System.IO
open System.Text.RegularExpressions

In [102]:
type Instruction = 
    | L 
    | R
    with 
    static member fromChar (c:Char) = 
        match c with 
        | 'L' -> L 
        | 'R' -> R
        | _ -> failwith $"Problem with input {c}"

In [46]:
type Node = Node of string * string 
    with
        member this.Item 
            with get(ins:Instruction) = 
                match this, ins with 
                | (Node (left,right)), L -> left 
                | (Node (left,right)), R -> right 



In [60]:
let testInput = """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)"""
let testInput2 = """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""

In [61]:
let instructions = 
    (testInput2.Split("\n")[0]).Trim() 
    |> Seq.map Instruction.fromChar
    |> Array.ofSeq

In [48]:
let parseLine (line:String) = 
    let g = Regex.Match(line.Trim(), @"(\w\w\w) = \((\w\w\w), (\w\w\w)\)").Groups
    (g[1].Value,Node (g[2].Value,g[3].Value))
parseLine "GGG = (GGG, GGG)"

(GGG, Node ("GGG", "GGG")) Item1 GGG Item2 Node ("GGG", "GGG") Item1 GGG Item2 GGG

In [62]:
let map = 
    testInput2.Split("\n")[2..]
    |> Array.map parseLine 
    |> Map.ofArray

In [63]:
let mutable currentSpot = "AAA" 
let mutable i = 0
let n = Array.length instructions
let mutable inst = instructions[i%n]
while currentSpot <> "ZZZ" && i<100 do 
    currentSpot <- map[currentSpot][inst]
    i <- i + 1
    inst <- instructions[i%n]


6

In [66]:
let input = File.ReadAllLines("input_8.txt")

In [68]:
let instructions = 
    (input[0]).Trim() 
    |> Seq.map Instruction.fromChar
    |> Array.ofSeq

In [69]:
let map = 
    input[2..]
    |> Array.map parseLine 
    |> Map.ofArray

In [73]:
let mutable currentSpot = "AAA" 
let mutable i = 0
let n = Array.length instructions
let mutable inst = instructions[i%n]
while currentSpot <> "ZZZ" && i>=0 do 
    currentSpot <- map[currentSpot][inst]
    i <- i + 1
    inst <- instructions[i%n]
i

19637

# Part 2

In [93]:
// Map nodes to integers
type NodeId = int
let nodeToNumber : Map<string, NodeId> =
    input[2..]
    |> Array.mapi (fun i line -> line |> parseLine |> fst , i )
    |> Map.ofArray
let numberToNode = 
    input[2..] 
    |> Array.map (parseLine >> fst)

let Ls : NodeId array= 
    input[2..]
    |> Array.map (fun line -> 
        let node = 
            parseLine line 
            |> snd
        nodeToNumber[node[L]])

let Rs : NodeId array=
    input[2..]
    |> Array.map (fun line -> 
        let node = 
            parseLine line 
            |> snd
        nodeToNumber[node[R]])

In [106]:
let endsWithZ (id: NodeId) = 
    numberToNode[id][2] = 'Z'
let endsWithA (id: NodeId) = 
    numberToNode[id][2] = 'A'

In [104]:
let step (node: NodeId) (inst: Instruction) = 
    match inst with 
    | L -> Ls[node]
    | R -> Rs[node]

In [109]:
let mutable currentSpots =
    [|0..Ls.Length-1|]
    |> Array.filter endsWithA
let mutable i = 0
let n = Array.length instructions
let mutable inst = instructions[i%n]
while not (Array.forall currentSpots endsWithA) && i>=0 do 
    for j in [0..currentSpots.Length - 1] do 
        currentSpots[j] <- step currentSpots[j] inst  

Error: input.fsx (3,5)-(3,12) parse error Unexpected keyword 'mutable' in expression
input.fsx (3,5)-(3,12) parse error Identifier expected